In [351]:
import pandas as pd

main_cities = [line.strip() for line in open("DATASET/main_cities")]

In [352]:
import re

def is_new_journey(string):
    regex_header = r"-- ([0-9]+) --"
    matches_it = re.finditer(regex_header, line, re.MULTILINE)
    matches = [m for m in matches_it]
    return len(matches) == 1

regex_line = r"^([0-9]{7}) (.{20}) .([0-9\ ]{5}) .([0-9\ ]{5})"
def is_line_timetable(string):
    matches_it = re.finditer(regex_line, line, re.MULTILINE)
    matches = [m for m in matches_it]
    return len(matches) == 1

def get_line_timetable(line):
    if is_line_timetable(line):
        matches_it = re.finditer(regex_line, line, re.MULTILINE)
        matches = [m for m in matches_it]
        match = matches[0]
        return match.groups()

    return None, None, None, None

def is_start_journey(string):
    return string.strip() == ""


cities = set()
links = []

old = None
with open("DATASET/FPLAN", "r") as f:
    for line in f:
        if is_new_journey(line):
            #print("\n New Journey \n")
            old=None
            pass
        
        if is_line_timetable(line):
            number, name, start_str, end_str = get_line_timetable(line)
            name = name.strip()

            if name in main_cities:             
                cities.add(name)
                if old != None:
                    if old < name:
                        links.append((old, name))
                    else:
                        links.append((name, old))
            

                old = name
                          
print(cities)
print(links)

{'Bern', 'Montreux', 'Locarno', 'Zürich HB', 'Sion', 'Thun', 'Fribourg/Freiburg', 'Luzern', 'Visp', 'Chur', 'Zug', 'Biel/Bienne', 'Winterthur', 'La Chaux-de-Fonds', 'Neuchâtel', 'Bulle', 'Vernier', 'Martigny', 'Uster', 'Lausanne', 'Basel SBB', 'Schaffhausen', 'Lugano', 'St. Gallen', 'Genève'}
[('Basel SBB', 'Zürich HB'), ('Basel SBB', 'Zürich HB'), ('Basel SBB', 'Zürich HB'), ('Basel SBB', 'Zürich HB'), ('Bern', 'Thun'), ('Basel SBB', 'Bern'), ('Basel SBB', 'Bern'), ('Bern', 'Thun'), ('Basel SBB', 'Zürich HB'), ('Basel SBB', 'Zürich HB'), ('Basel SBB', 'Zürich HB'), ('Basel SBB', 'Zürich HB'), ('Lugano', 'Zug'), ('Zug', 'Zürich HB'), ('Zug', 'Zürich HB'), ('Lugano', 'Zug'), ('Lugano', 'Zug'), ('Zug', 'Zürich HB'), ('Zug', 'Zürich HB'), ('Lugano', 'Zug'), ('Lugano', 'Zug'), ('Zug', 'Zürich HB'), ('Zug', 'Zürich HB'), ('Lugano', 'Zug'), ('Lugano', 'Zug'), ('Zug', 'Zürich HB'), ('Zug', 'Zürich HB'), ('Lugano', 'Zug'), ('Lugano', 'Zug'), ('Zug', 'Zürich HB'), ('Zug', 'Zürich HB'), ('Lugano

In [353]:
from collections import Counter
count_dict = dict(Counter(links))
count_dict

{('Basel SBB', 'Zürich HB'): 245,
 ('Bern', 'Thun'): 345,
 ('Basel SBB', 'Bern'): 87,
 ('Lugano', 'Zug'): 79,
 ('Zug', 'Zürich HB'): 511,
 ('Montreux', 'Sion'): 10,
 ('Lausanne', 'Montreux'): 240,
 ('Genève', 'Lausanne'): 310,
 ('Thun', 'Visp'): 89,
 ('Chur', 'Zürich HB'): 112,
 ('Basel SBB', 'Luzern'): 115,
 ('Lugano', 'Luzern'): 30,
 ('Schaffhausen', 'Zürich HB'): 132,
 ('St. Gallen', 'Winterthur'): 165,
 ('Winterthur', 'Zürich HB'): 742,
 ('Biel/Bienne', 'Neuchâtel'): 171,
 ('Genève', 'Neuchâtel'): 38,
 ('Biel/Bienne', 'Zürich HB'): 111,
 ('Lausanne', 'Neuchâtel'): 83,
 ('Locarno', 'Luzern'): 4,
 ('Bern', 'Fribourg/Freiburg'): 195,
 ('Fribourg/Freiburg', 'Lausanne'): 81,
 ('Bern', 'Zürich HB'): 172,
 ('Lugano', 'Zürich HB'): 9,
 ('Basel SBB', 'Biel/Bienne'): 35,
 ('Martigny', 'Sion'): 215,
 ('Martigny', 'Montreux'): 138,
 ('Sion', 'Visp'): 197,
 ('Luzern', 'Zug'): 436,
 ('Bern', 'Luzern'): 44,
 ('Chur', 'St. Gallen'): 30,
 ('Bern', 'Biel/Bienne'): 160,
 ('Biel/Bienne', 'La Chaux-de-

In [354]:
from datetime import date

regex_geo = r"^[0-9]{7} +([0-9\.\-]+)  +([0-9\.\-]+) .+\% (.+)"
def get_groups(string, regex): 
    matches_it = re.finditer(regex, string, re.MULTILINE)
    matches = [m for m in matches_it]
    match = matches[0]
    return match.groups()

cities_to_point = pd.DataFrame([], columns=["name", "long","lat", "date", "radius"])
with open("DATASET/BFKOORD_GEO", "r") as f:
    for line in f: 
        y,x,name = get_groups(line, regex_geo)
        if name in main_cities:
            cities_to_point = cities_to_point.append({"name" : name, "lat" : float(x), "long" :float(y), "date": date.today(), "radius" : 10}, ignore_index=True)
        
cities_to_point
map_name_to_coor = {r[1][0] : (r[1][1], r[1][2])for r in cities_to_point.iterrows()}
print(map_name_to_coor)

{'Basel SBB': (7.589551, 47.547405), 'Vernier': (6.093894, 46.22071), 'Genève': (6.142452, 46.210203), 'Lausanne': (6.629093, 46.516779), 'Montreux': (6.910432, 46.435875), 'Martigny': (7.079122, 46.105828), 'Sion': (7.359191, 46.227546), 'Visp': (7.881458, 46.294025), 'Zug': (8.515295, 47.173624), 'Zürich HB': (8.540192, 47.378177), 'Uster': (8.718688, 47.350367), 'Schaffhausen': (8.632728, 47.698282), 'Bulle': (7.055037, 46.618189), 'Fribourg/Freiburg': (7.151023, 46.803268), 'Neuchâtel': (6.935705, 46.996723), 'Biel/Bienne': (7.242908, 47.132893), 'La Chaux-de-Fonds': (6.826018, 47.098644), 'Luzern': (8.31017, 47.05017), 'Lugano': (8.946874, 46.005506), 'Locarno': (8.801358, 46.172428), 'Winterthur': (8.723799, 47.500334), 'St. Gallen': (9.369877, 47.42318), 'Bern': (7.439118, 46.948825), 'Thun': (7.629594, 46.754854), 'Chur': (9.528925, 46.85308)}


In [355]:
links_df =[{"name": k, "weight": v, "coordinates" :[map_name_to_coor[c] for c in k]} for k,v in count_dict.items()]


from geojson import LineString, Feature, FeatureCollection
coll = FeatureCollection([Feature(geometry=LineString(l["coordinates"]), properties={"weight": l["weight"]}) for l in links_df])



In [356]:
from mapboxgl.viz import *
from mapboxgl.utils import *
token = (open(".mapbox_access_token").read().strip())
points = df_to_geojson(cities_to_point, lat='lat', lon='long', precision=3)

print(points)

color_breaks = [0,9,100,1000,10000]
color_stops = create_color_stops(color_breaks, colors='YlGnBu')
# Create the viz from the dataframe
viz = CircleViz(points,
                access_token=token,
                stroke_color='black',
                stroke_width=0.5,
                radius=4,
                center=(8, 47),
                zoom=7,
                style="mapbox://styles/harne/ck50szq0t0hpl1cmx9hxbfz7h")
viz.show()

{"features": [{"geometry": {"coordinates": [7.59, 47.547], "type": "Point"}, "properties": {"date": 1578268800, "name": "Basel SBB", "radius": 10}, "type": "Feature"}, {"geometry": {"coordinates": [6.094, 46.221], "type": "Point"}, "properties": {"date": 1578268800, "name": "Vernier", "radius": 10}, "type": "Feature"}, {"geometry": {"coordinates": [6.142, 46.21], "type": "Point"}, "properties": {"date": 1578268800, "name": "Gen\u00e8ve", "radius": 10}, "type": "Feature"}, {"geometry": {"coordinates": [6.629, 46.517], "type": "Point"}, "properties": {"date": 1578268800, "name": "Lausanne", "radius": 10}, "type": "Feature"}, {"geometry": {"coordinates": [6.91, 46.436], "type": "Point"}, "properties": {"date": 1578268800, "name": "Montreux", "radius": 10}, "type": "Feature"}, {"geometry": {"coordinates": [7.079, 46.106], "type": "Point"}, "properties": {"date": 1578268800, "name": "Martigny", "radius": 10}, "type": "Feature"}, {"geometry": {"coordinates": [7.359, 46.228], "type": "Point"}

In [367]:
print(coll)
viz = LinestringViz(coll,
            access_token=token,
            center=(8, 47),
            zoom=7,
            line_width_property='weight',
            line_width_stops=create_numeric_stops([0, 20, 50, 100, 200,500,1000,  1500], 0, 20),   
            color_property='weight',
            color_stops=create_color_stops([0, 20, 50, 100, 200,500,1000,  1500], colors='Blues'),
            opacity = 0.8,
            style="mapbox://styles/harne/ck50szq0t0hpl1cmx9hxbfz7h",
            box_zoom_on = False,
            )
viz.show()

{"features": [{"geometry": {"coordinates": [[7.589551, 47.547405], [8.540192, 47.378177]], "type": "LineString"}, "properties": {"weight": 245}, "type": "Feature"}, {"geometry": {"coordinates": [[7.439118, 46.948825], [7.629594, 46.754854]], "type": "LineString"}, "properties": {"weight": 345}, "type": "Feature"}, {"geometry": {"coordinates": [[7.589551, 47.547405], [7.439118, 46.948825]], "type": "LineString"}, "properties": {"weight": 87}, "type": "Feature"}, {"geometry": {"coordinates": [[8.946874, 46.005506], [8.515295, 47.173624]], "type": "LineString"}, "properties": {"weight": 79}, "type": "Feature"}, {"geometry": {"coordinates": [[8.515295, 47.173624], [8.540192, 47.378177]], "type": "LineString"}, "properties": {"weight": 511}, "type": "Feature"}, {"geometry": {"coordinates": [[6.910432, 46.435875], [7.359191, 46.227546]], "type": "LineString"}, "properties": {"weight": 10}, "type": "Feature"}, {"geometry": {"coordinates": [[6.629093, 46.516779], [6.910432, 46.435875]], "type"

In [358]:
viz.__dict__

{'access_token': 'pk.eyJ1IjoiaGFybmUiLCJhIjoiY2oxejQ0Yzd1MDFleDJ3azJzem5tZzRyeCJ9.fXoksm1YeRxCflOpeCMpNg',
 'data': {"features": [{"geometry": {"coordinates": [[7.589551, 47.547405], [8.540192, 47.378177]], "type": "LineString"}, "properties": {"weight": 245}, "type": "Feature"}, {"geometry": {"coordinates": [[7.439118, 46.948825], [7.629594, 46.754854]], "type": "LineString"}, "properties": {"weight": 345}, "type": "Feature"}, {"geometry": {"coordinates": [[7.589551, 47.547405], [7.439118, 46.948825]], "type": "LineString"}, "properties": {"weight": 87}, "type": "Feature"}, {"geometry": {"coordinates": [[8.946874, 46.005506], [8.515295, 47.173624]], "type": "LineString"}, "properties": {"weight": 79}, "type": "Feature"}, {"geometry": {"coordinates": [[8.515295, 47.173624], [8.540192, 47.378177]], "type": "LineString"}, "properties": {"weight": 511}, "type": "Feature"}, {"geometry": {"coordinates": [[6.910432, 46.435875], [7.359191, 46.227546]], "type": "LineString"}, "properties": {"w